<a href="https://colab.research.google.com/github/br0ast/KRaE-lab/blob/main/RDFLib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.1 MB/s eta 0:00:00


In [2]:
from rdflib import *

In [5]:
examplegraph = Graph() #create a graph

In [6]:
namespacestring = "https://www.example.org/" #create a string with a namespace
# like http://dbpedia.org/resource/ in dbpedia
example = Namespace(namespacestring) #create a Namespace with the Namespace constructor method

In [7]:
examplegraph.bind("ex", example) #add the prefix to the graph

In [8]:
examplegraph.add((example.Bruno, example.isTutorOf, example.DHDK)) # adding a triple to the graph, the add function takes a tuple with 3 elements 
#(subject, property, object) as an input, and adds the triple to the graph

<Graph identifier=N857908104ad64fc3bab4ccb98a2255d8 (<class 'rdflib.graph.Graph'>)>

In [9]:
examplegraph.add((example.Bruno, example.hasRole, example.Tutor))

<Graph identifier=N857908104ad64fc3bab4ccb98a2255d8 (<class 'rdflib.graph.Graph'>)>

In [10]:
examplegraph.add((example.Bruno, example.hasRole, example.PhDStudent))

<Graph identifier=N857908104ad64fc3bab4ccb98a2255d8 (<class 'rdflib.graph.Graph'>)>

In [11]:
examplegraph.add((example.Bruno, RDFS.label, Literal("Bruno"))) #Literal is used to create a literal to add to the graph, you can't add strings directly

<Graph identifier=N857908104ad64fc3bab4ccb98a2255d8 (<class 'rdflib.graph.Graph'>)>

In [12]:
examplegraph.add((example.Bruno, example.livesIn, URIRef("http://dbpedia.org/resource/Munich"))) #URIRef is used when the URI of the element you are
# adding is not bound to the graph or when you iterate to get multiple triples at the same time

<Graph identifier=N857908104ad64fc3bab4ccb98a2255d8 (<class 'rdflib.graph.Graph'>)>

In [13]:
print(examplegraph.serialize(format="ttl"))

@prefix ex: <https://www.example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Bruno rdfs:label "Bruno" ;
    ex:hasRole ex:PhDStudent,
        ex:Tutor ;
    ex:isTutorOf ex:DHDK ;
    ex:livesIn <http://dbpedia.org/resource/Munich> .




In [14]:
dbr = Namespace("http://dbpedia.org/resource/")

In [15]:
examplegraph.bind("dbr", dbr)

In [16]:
print(examplegraph.serialize(format="ttl"))

@prefix dbr: <http://dbpedia.org/resource/> .
@prefix ex: <https://www.example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Bruno rdfs:label "Bruno" ;
    ex:hasRole ex:PhDStudent,
        ex:Tutor ;
    ex:isTutorOf ex:DHDK ;
    ex:livesIn dbr:Munich .




In [68]:
automaticallyconvertedgraph = Graph()
automaticallyconvertedgraph.bind("ex", example)
automaticallyconvertedgraph.bind("dbr", dbr)

In [69]:
startingdata = {"ex:Bruno":{"literals":{"ex:name":"Bruno", "ex:age":27}, "uris":{"ex:hasRole":["ex:PhDStudent", "ex:Tutor"], "ex:isTutorOf":"ex:DHDK", "ex:livesIn":"dbr:Munich"}},
                "ex:Valentina":{"literals":{"ex:name":"Valentina", "ex:age":27}, "uris":{"ex:hasRole":["ex:PhDStudent", "ex:Tutor"], "ex:isTutorOf":"ex:DHDK", "ex:livesIn":["dbr:Graz", "dbr:Bologna"]}}}

In [70]:
from rdflib.namespace import XSD
dbpediauri = "http://dbpedia.org/resource/"
for person in startingdata: #iterate over our starting data, "person" is either ex:Bruno or ex:Valentina
  if "ex:" in person: #checking whether we need to use the example namespace or dbpedia
    personuri = URIRef(person.replace("ex:", namespacestring)) # we replace the string with a uri that uses the namespace of example + the name without "ex:"
    for literal in startingdata[person]["literals"]: #we check the literals
      if type(startingdata[person]["literals"][literal]) == str: #if the literal is a string
        lit = startingdata[person]["literals"][literal]
        automaticallyconvertedgraph.add((personuri, URIRef(literal.replace("ex:", namespacestring)), Literal(lit))) #we add the literal using the property ex:...
      elif type(startingdata[person]["literals"][literal]) == int: #if the literal is an integer
        lit = startingdata[person]["literals"][literal]
        automaticallyconvertedgraph.add((personuri, URIRef(literal.replace("ex:", namespacestring)), Literal(lit, datatype="XSD.integer"))) # we specify the datatype using datatype="XSD.integer"
    for uri in startingdata[person]["uris"]: #now we look at object properties (the object of an object property will be a uri and not a literal)
      if type(startingdata[person]["uris"][uri]) == str: #we check if it is a string
        uristring = startingdata[person]["uris"][uri]
        if "ex:" in uristring: # check whether we need to use the example namespace
          uristring = uristring.replace("ex:", namespacestring) # we make the uri of the object
        if "dbr:" in uristring: # check whether we need to use the dbpedia namespace
          uristring = uristring.replace("dbr:", dbpediauri) # we make the uri of the object
        automaticallyconvertedgraph.add((personuri, URIRef(uri.replace("ex:", namespacestring)), URIRef(uristring))) #we add the property, creating a uri by replacing the ex: with the namespace, and create the triple
      else:
        for multiobject in startingdata[person]["uris"][uri]: # if it is not a string than it means it is a list with multiple object so we iterate over the object of the list
          if "ex:" in multiobject:
            uristring = multiobject.replace("ex:", namespacestring) # we create the uri for each object
          elif "dbr:" in multiobject:
            uristring = multiobject.replace("dbr:", dbpediauri)
          automaticallyconvertedgraph.add((personuri, URIRef(uri.replace("ex:", namespacestring)), URIRef(uristring))) # and then add the triple by replacing


In [71]:
print(automaticallyconvertedgraph.serialize(format="ttl"))

@prefix dbr: <http://dbpedia.org/resource/> .
@prefix ex: <https://www.example.org/> .

ex:Bruno ex:age "27"^^<XSD.integer> ;
    ex:hasRole ex:PhDStudent,
        ex:Tutor ;
    ex:isTutorOf ex:DHDK ;
    ex:livesIn dbr:Munich ;
    ex:name "Bruno" .

ex:Valentina ex:age "27"^^<XSD.integer> ;
    ex:hasRole ex:PhDStudent,
        ex:Tutor ;
    ex:isTutorOf ex:DHDK ;
    ex:livesIn dbr:Bologna,
        dbr:Graz ;
    ex:name "Valentina" .




In [72]:
for s,p,o in automaticallyconvertedgraph.triples((None, None, None)): #iterates over triples without specifing any 
  print(s,p,o)

https://www.example.org/Valentina https://www.example.org/livesIn http://dbpedia.org/resource/Bologna
https://www.example.org/Bruno https://www.example.org/hasRole https://www.example.org/PhDStudent
https://www.example.org/Valentina https://www.example.org/livesIn http://dbpedia.org/resource/Graz
https://www.example.org/Bruno https://www.example.org/name Bruno
https://www.example.org/Bruno https://www.example.org/livesIn http://dbpedia.org/resource/Munich
https://www.example.org/Valentina https://www.example.org/age 27
https://www.example.org/Valentina https://www.example.org/hasRole https://www.example.org/PhDStudent
https://www.example.org/Bruno https://www.example.org/age 27
https://www.example.org/Valentina https://www.example.org/isTutorOf https://www.example.org/DHDK
https://www.example.org/Valentina https://www.example.org/name Valentina
https://www.example.org/Bruno https://www.example.org/hasRole https://www.example.org/Tutor
https://www.example.org/Valentina https://www.examp

In [73]:
for s,p,o in automaticallyconvertedgraph.triples((None, example.hasRole, None)): #iterates over triples that have a specific property
  print(s,p,o)

https://www.example.org/Bruno https://www.example.org/hasRole https://www.example.org/PhDStudent
https://www.example.org/Valentina https://www.example.org/hasRole https://www.example.org/PhDStudent
https://www.example.org/Bruno https://www.example.org/hasRole https://www.example.org/Tutor
https://www.example.org/Valentina https://www.example.org/hasRole https://www.example.org/Tutor


In [74]:
for s,p,o in automaticallyconvertedgraph.triples((example.Valentina, None, None)):
  print(s,p,o) #iterates over triples that have a specific subject

https://www.example.org/Valentina https://www.example.org/name Valentina
https://www.example.org/Valentina https://www.example.org/age 27
https://www.example.org/Valentina https://www.example.org/hasRole https://www.example.org/PhDStudent
https://www.example.org/Valentina https://www.example.org/hasRole https://www.example.org/Tutor
https://www.example.org/Valentina https://www.example.org/isTutorOf https://www.example.org/DHDK
https://www.example.org/Valentina https://www.example.org/livesIn http://dbpedia.org/resource/Graz
https://www.example.org/Valentina https://www.example.org/livesIn http://dbpedia.org/resource/Bologna


In [75]:
querytext = ''' 
prefix ex: <https://www.example.org/>
select distinct ?people where { ?people ex:hasRole ?role}
''' #we create the sparql query
result = automaticallyconvertedgraph.query(querytext) #you can query them too
for el in result:
  print(el["people"]) # we print the result, the "people" string is the same as the ?people in the query, if you put another variable in the query you need to change it here, without typing "?" in the results part

https://www.example.org/Bruno
https://www.example.org/Valentina


In [25]:
pip install SPARQLWrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://dbpedia.org/sparql/"

query = """
prefix dbr: <http://dbpedia.org/resource/> 
SELECT ?city ?country
WHERE {VALUES ?city {dbr:Munich dbr:Graz} ?city dbo:country ?country}
"""

sparql = SPARQLWrapper(endpoint_url)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["country"]["value"])


http://dbpedia.org/resource/Austria
http://dbpedia.org/resource/http://dbpedia.org/resource/Germany


In [77]:
dbo = Namespace("http://dbpedia.org/ontology/")
automaticallyconvertedgraph.bind("dbo", dbo)
for result in results["results"]["bindings"]:
  if "Germany" not in result["country"]["value"]:
    automaticallyconvertedgraph.add((URIRef(result["city"]["value"]), dbo.Country, URIRef(result["country"]["value"])))
  else:
    automaticallyconvertedgraph.add((URIRef(result["city"]["value"]), dbo.Country, URIRef(result["country"]["value"].replace("http://dbpedia.org/resource/", "", 1))))


In [78]:
print(automaticallyconvertedgraph.serialize(format="ttl"))

@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix dbr: <http://dbpedia.org/resource/> .
@prefix ex: <https://www.example.org/> .

ex:Bruno ex:age "27"^^<XSD.integer> ;
    ex:hasRole ex:PhDStudent,
        ex:Tutor ;
    ex:isTutorOf ex:DHDK ;
    ex:livesIn dbr:Munich ;
    ex:name "Bruno" .

ex:Valentina ex:age "27"^^<XSD.integer> ;
    ex:hasRole ex:PhDStudent,
        ex:Tutor ;
    ex:isTutorOf ex:DHDK ;
    ex:livesIn dbr:Bologna,
        dbr:Graz ;
    ex:name "Valentina" .

dbr:Graz dbo:Country dbr:Austria .

dbr:Munich dbo:Country dbr:Germany .




In [79]:
#save the graph

automaticallyconvertedgraph.serialize(destination="brunovalentinagraph.ttl", format="ttl")

<Graph identifier=Nd037bad85ffe4c7f9e6976fd260b7315 (<class 'rdflib.graph.Graph'>)>